In [1]:
def rename_from_fn_to_slice_numbers( image_list, sorted_fn_list, testing=True):
    # RENAME FROM FILENAMES TO SLICE NAMES
    for image_full_name in image_list:
        for sorted_fn_line in sorted_fn_list:
            fn, slice_num = sorted_fn_line.split(' ')
            if fn=='Placeholder':
                print(slice_num)

            if fn in image_full_name:
                # Slice number need to be preceeded by zeros to be 3 digits
                if len(slice_num)==1:
                    slice_num = '00'+slice_num
                elif len(slice_num)==2:
                    slice_num = '0'+slice_num

                old_designation = image_path+image_full_name
                new_designation = image_path+stack+'_slice_'+slice_num+'.jpg'
                print(old_designation)
                print(new_designation)
                
                if not testing:
                    ! cp $old_designation $new_designation


def insert_dummy_placeholder_images( image_list, sorted_fn_list, testing=True):
    # FILL IN PLACEHOLDER IMAGES WITH THE PREVIOUS IMAGE RENAMED
    prev_slice_num = -0.1

    for image_full_name in image_list:
        img_slice_num = image_full_name.split('_')[2][:image_full_name.split('_')[2].index('.')]

        # Iterate through every fn/slice number in sorted filenames
        for sorted_fn_line in sorted_fn_list:
            fn_sfns, slice_num_sfns = sorted_fn_line.split(' ')
            if len(slice_num_sfns)==1:
                slice_num_sfns = '00'+slice_num_sfns
            elif len(slice_num_sfns)==2:
                slice_num_sfns = '0'+slice_num_sfns

            # If there is a placeholder slice after the current img_slice_num, then copy
            if int(slice_num_sfns)==int(img_slice_num)+1 and fn_sfns=='Placeholder':
                prev_img_fn = stack+'_slice_'+str(img_slice_num)+'.jpg'
                placeholder_img_fn = stack+'_slice_'+slice_num_sfns+'_placeholder_cfp.jpg'
                print(prev_img_fn)
                print(placeholder_img_fn)
                
                if not testing:
                    ! cp $images_local/$prev_img_fn $images_local/$placeholder_img_fn
                    
import os
import sys
import json

sys.path.append(os.path.join( os.environ['NG_REPO_DIR'], 'src', 'utilities'))
from filepath_manager import *
from precomputing_utilities import *

atlas_root = os.environ['ATLAS_DATA_ROOT_DIR']

In [2]:
stack = 'MD585'

# Filepaths to image files already on the computer
#    NOT NECESSARY as long as the image files are in the proper location
#    (check using `get_image_fp()`)
atlas_raw_jpeg_image_path = atlas_root+'CSHL_data_processed/'+stack+'/'+stack+'_prep2_raw_grayJpeg/'
atlas_sorted_fn_path = atlas_root+'CSHL_data_processed/'+stack+'/'+stack+'_sorted_filenames.txt'


# Input image files for precomputing script
temp_renamed_image_path = get_image_fp( stack, precomputed=False, brain_crop='brainstem', \
                                     image_version='grayJpeg', resolution='raw' )
precomputed_input_image_path = get_image_fp( stack, precomputed=False, brain_crop='brainstem', \
                                     image_version='grayJpeg', resolution='raw' )
temp_sorted_fn_path = precomputed_input_image_path+'../'+stack+'_sorted_filenames.txt'

# Output files from precomputing script
precomputed_output_image_path = get_image_fp( stack, precomputed=True, brain_crop='brainstem', \
                                     image_version='grayJpeg', resolution='raw' )

In [3]:
data = {
    "s3_creds_file": "/home/alexn/Documents/json_credentials/alex_aws_credentials.json",
    "s3_bucket_name_for_download": "mousebrainatlas-data",
    "sorted_filename": stack+"_sorted_filenames.txt",
    "ext": ".jpg",
    "folder_to_write_to": "/mnt/x/temp_s3_download/"+stack+"_prep2_lossless_grayJpeg",
    "folder_to_convert_from": precomputed_input_image_path,
    "folder_to_convert_to": precomputed_output_image_path,
    "s3_bucket_name_for_upload": "mousebrainatlas-datajoint-jp2k",
    "s3_dir_to_write_to": "precomputed/"+stack+"_fullres",
    "voxel_resolution": [460, 460, 20000],
    "voxel_offset": [0, 0, 0],
    "overwrite": True
}

with open('input.json', 'w') as file:
    json.dump(data, file)
  
 # RUN PRECOMPUTING SCRIPT
# running in terminal is much cleaner
#! python ../src/precomputation/precompute_images_v1.py input.json

In [ ]:
images_local = data["folder_to_convert_from"]
precomputed_local = data["folder_to_convert_to"]
precomputed_s3 = 's3://mousebrainatlas-datajoint-jp2k/'+s3_precomputed_path_relative

with open( sorted_fn_path, 'r') as sfns:
    sorted_fn_list = sfns.read().split('\n')[:-1]
    
image_list = ! ls $images_local

In [4]:
# RUN PRECOMPUTING SCRIPT
# running in terminal is much cleaner
######! python neuroglancer_utilities.py input.json;
#! python ../src/precompute_images_v1.py input.json

In [5]:
# COPY PRECOMPUTED FILES TO S3
# running in terminal is much cleaner
#! aws s3 cp --recursive $precomputed_local $precomputed_s3